In [1]:
import pandas as pd
import sqlite3

In [2]:
# ==============================================================================
# STEP 2: DATA INGESTION & CLEANING (ETL Process)
# ==============================================================================

# Ingestion: Load the raw dataset with specific encoding handling
# Note: The UCI Online Retail dataset typically uses 'ISO-8859-1' encoding.
file_path = '../data/online_retail_II.csv' 
df = pd.read_csv(file_path, encoding='ISO-8859-1')

print(f"Raw Data Volume: {df.shape[0]} rows")

Raw Data Volume: 1067371 rows


In [3]:
# Data Cleaning: Remove records with missing Customer IDs (Essential for user segmentation)
df = df.dropna(subset=['Customer ID'])

In [4]:
# Data Integrity: Ensure valid positive values for Quantity and Price
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]

In [5]:
# Type Conversion: Convert 'InvoiceDate' to datetime objects for time-series analysis
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [6]:
# Feature Engineering: Calculate 'TotalLinePrice' (Revenue per line item)
df['TotalLinePrice'] = df['Quantity'] * df['Price']

print(f"Cleaned Data Volume: {df.shape[0]} rows")
print("✔ ETL Pipeline completed successfully.")

Cleaned Data Volume: 805549 rows
✔ ETL Pipeline completed successfully.


In [7]:
# ==============================================================================
# STEP 3: SQL ENGINE SETUP (In-Memory Database)
# ==============================================================================

# Database Connection: Initialize an in-memory SQLite database for rapid querying
conn = sqlite3.connect(':memory:')

In [8]:
# Data Loading: Dump the transformed Pandas DataFrame into a SQL table named 'retail'
# 'index=False' prevents writing the Pandas index as a column
df.to_sql('retail', conn, index=False, if_exists='replace')

print("✔ SQL Engine initialized. Data loaded into table 'retail'.")

✔ SQL Engine initialized. Data loaded into table 'retail'.


In [9]:
# ==============================================================================
# STEP 4: DATA ANALYSIS (SQL Querying)
# ==============================================================================

# Business Insight: Identify the top 5 best-selling products by total revenue
query_top_products = """
SELECT 
    Description, 
    SUM(Quantity) as Total_Units_Sold,
    SUM(TotalLinePrice) as Total_Revenue
FROM retail
GROUP BY Description
ORDER BY Total_Revenue DESC
LIMIT 5;
"""

In [10]:
# Execution: Run the SQL query and load results into a DataFrame
top_products_df = pd.read_sql(query_top_products, conn)

print("\n--- Top 5 Products by Revenue ---")
print(top_products_df)


--- Top 5 Products by Revenue ---
                          Description  Total_Units_Sold  Total_Revenue
0            REGENCY CAKESTAND 3 TIER             24899      286486.30
1  WHITE HANGING HEART T-LIGHT HOLDER             93640      252072.46
2         PAPER CRAFT , LITTLE BIRDIE             80995      168469.60
3                              Manual              9803      152340.57
4             JUMBO BAG RED RETROSPOT             75759      136980.08
